In [48]:
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.metrics.pairwise import cosine_similarity, euclidean_distances
import warnings
warnings.filterwarnings('ignore')
#MIGHT HAVE TO ENCODE THE BINNING OF THE LENGTHS SO THIS CAN BE A 
#PARAMETER FOR A COLD START

In [49]:
US_trails = pd.read_csv('../data/US_trails_engineered.csv')

In [50]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'starVotes', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range'],
      dtype='object')

In [51]:
US_trails.shape

(20691, 19)

In [52]:
US_trails.dtypes

name                   object
location               object
difficulty             object
length                float64
ascent                  int64
descent                 int64
stars                 float64
starVotes               int64
latitude              float64
longitude             float64
summary                object
url                    object
difficulty_encoded      int64
category               object
type_Featured Ride      int64
type_Trail              int64
city/town              object
state                  object
length_range           object
dtype: object

In [53]:
#I don't need the latitude and longitude columns since I'll be filtering on those
columns_to_scale = list(US_trails.select_dtypes(exclude=['object']).columns.difference(['stars','starVotes',
                        'latitude','longitude','type_Featured Ride', 'type_Trail']))
columns_to_scale

['ascent', 'descent', 'difficulty_encoded', 'length']

In [54]:
#this scales each column individually, BUT REALLY WHAT IS IT DOING?
for col in columns_to_scale:
    US_trails[col + "_scaled"] = preprocessing.scale(US_trails[col])

In [55]:
'''
#THIS DOES THE SAME AS ABOVE, EXCEPT THAT IT REWRITES ALL THE QUANTITATIVE COLUMNS WITH THEIR SCALED VALUES
scaler = preprocessing.StandardScaler()
scaler.fit(US_trails[quantitative_columns])
US_trails[quantitative_columns] = scaler.transform(US_trails[quantitative_columns])
US_trails.head()
'''

'\n#THIS DOES THE SAME AS ABOVE, EXCEPT THAT IT REWRITES ALL THE QUANTITATIVE COLUMNS WITH THEIR SCALED VALUES\nscaler = preprocessing.StandardScaler()\nscaler.fit(US_trails[quantitative_columns])\nUS_trails[quantitative_columns] = scaler.transform(US_trails[quantitative_columns])\nUS_trails.head()\n'

In [56]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'starVotes', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range', 'ascent_scaled', 'descent_scaled',
       'difficulty_encoded_scaled', 'length_scaled'],
      dtype='object')

In [57]:
scaled_cols = [col for col in US_trails.columns if 'scaled' in col]
scaled_cols
#US_trails_scaled = US_trails.filter(regex='scaled')
#US_trails_scaled.head()

['ascent_scaled',
 'descent_scaled',
 'difficulty_encoded_scaled',
 'length_scaled']

In [58]:
Big_X = US_trails[scaled_cols].values
Big_X

array([[ 2.948716  , -2.79569706,  2.40444594,  2.09771539],
       [-0.09257181,  0.12246069,  1.6195909 , -0.07002184],
       [-0.1825315 ,  0.20147031, -0.7349742 , -0.07002184],
       ..., 
       [ 0.86338537, -0.22996377, -0.7349742 ,  1.80477792],
       [-0.12617121, -0.05635054, -0.7349742 ,  0.53928809],
       [ 2.32333359, -1.441098  ,  0.04988084,  1.53527546]])

In [59]:
#this code is from the web app
def cos_sim_recs(index, desired_state=None, desired_city_town=None):
    trail = X[index].reshape(1,-1)
    cs = cosine_similarity(trail, X)
    rec_index = np.argsort(cs)[0][::-1][1:]
    ordered_df = df.loc[rec_index]
    if desired_state is not None:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
    if desired_city_town is not None:
        ordered_df = ordered_df[ordered_df['city/town']== desired_city_town]
    #rec_df = ordered_df.head(n)
    rec_df = ordered_df.head(20)
    rec_df = rec_df.reset_index(drop=True)
    rec_df.index = rec_df.index+1 #this makes it so that there's no index 0 shown
    orig_row = df.loc[[index]].rename(lambda x: 'original')
    total = pd.concat([orig_row, rec_df])
    columns_to_output = ['name', 'location', 'difficulty', 'stars', 'length', 'ascent', 'descent',
       'category', 'summary', 'url']
    output = total[columns_to_output]
    return output

In [60]:
#this function will take in a state you ridden in, a trail you rode on, and the state you want to ride in and recommend
def cos_sim_recommendations(trail_name, state_name, scaled_features, desired_state = None, desired_city_town=None):
    index = US_trails.index[(US_trails['name'] == trail_name) & (US_trails['state']==state_name)][0]
    #index is the index of the given trail in the main df based on it's name and state
    trail = scaled_features[index].reshape(1,-1)
    print('The scaled features of the original trail are:{0}'.format(trail))
    #trail is the scaled features of the given trail reshaped into a 1 row array
    cs = cosine_similarity(trail, scaled_features)
    #cs is the array of cosine similarity measurements of the trail compared to the scaled_features of the other trails
    rec_index = np.argsort(cs)[0][::-1][1:]
    #rec_index is an array of the indices of the cosine similarity measurements ordered by greatest to least, leaving out the given trail
    print('The sorted cosine similarity measurements are:{0}'.format(np.sort(cs[0])[::-1])) #.sort(reverse = True)))
    print('The trail most similiar to the original trail has features: {0}'.format(scaled_features[rec_index][0]))
    ordered_df = US_trails.loc[rec_index]
    print('The trail most similar to the original trail is: {0}'.format(ordered_df.iloc[0]['name']))
    #ordered_df gives the trails of US_trails ordered by the trails most similar to least similar
    if desired_state is not None:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
    #if a desired state to ride in is given, the dataframe is returned with trails only in that state
    if desired_city_town is not None:
        ordered_df = ordered_df[ordered_df['city/town']== desired_city_town]
    #if a desired location to ride in is given, the dataframe is returned with trails only in that location
    rec_df = ordered_df.head(20)
    #rec_df is a dataframe with the number of desired trails to be recommended
    orig_row = US_trails.loc[[index]].rename(lambda x: 'original')
    #orig_row renames the index of the original trail given as 'original'
    total = pd.concat([orig_row, rec_df])
    #this combines the original row labeled as original row and the n recommended trails that are most similar to the original
    return total
    

In [61]:
#this function will take in a state you ridden in, a trail you rode on, and the state you want to ride in and recommend
def euclidean_dist_recommendations(trail_name, state_name, scaled_features, desired_state = None, desired_city_town= None):
    index = US_trails.index[(US_trails['name'] == trail_name) & (US_trails['state']==state_name)][0]
    #index is the index of the given trail in the main df based on it's name and state
    trail = scaled_features[index].reshape(1,-1)
    print('The scaled features of the original trail are:{0}'.format(trail))
    #trail is the scaled features of the given trail reshaped into a 1 row array
    ed = euclidean_distances(trail, scaled_features)
    #ed is the array of euclidean distance measurements of the trail compared to the scaled_features of the other trails
    #rec_index = np.argsort(ed)[0][::-1][1:]
    rec_index = np.argsort(ed)[0][1:]
    print('The sorted euclidean distance measurements are:{0}'.format(np.sort(ed[0]))) #.sort(reverse = True)))
    print('The trail most similiar to the original trail has features: {0}'.format(scaled_features[rec_index][0]))
    #rec_index is an array of the indices of the euclidean distance measurements ordered by greatest to least, leaving out the given trail
    ordered_df = US_trails.loc[rec_index]
    print('The trail most similar to the original trail is: {0}'.format(ordered_df.iloc[0]['name']))
    #ordered_df gives the trails of US_trails ordered by the trails most similar to least similar
    if desired_state is not None:
        ordered_df = ordered_df[ordered_df['state']== desired_state]
    #if a desired state to ride in is given, the dataframe is returned with trails only in that state
    if desired_city_town is not None:
        ordered_df = ordered_df[ordered_df['city/town']== desired_city_town]
    rec_df = ordered_df.head(20)
    #rec_df is a dataframe with the number of desired trails to be recommended
    orig_row = US_trails.loc[[index]].rename(lambda x: 'original')
    #orig_row renames the index of the original trail given as 'original'
    total = pd.concat([orig_row, rec_df])
    #this combines the original row labeled as original row and the n recommended trails that are most similar to the original
    return total

In [62]:
cos_sim_recommendations("The Whole Enchilada", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.90565559 -7.60488757  1.6195909   3.51553271]]
The sorted cosine similarity measurements are:[ 1.          0.99663965  0.99307429 ..., -0.9580581  -0.95849646
 -0.9604833 ]
The trail most similiar to the original trail has features: [ 0.35289123 -3.39450677  0.83473587  1.91023541]
The trail most similar to the original trail is: Whole Uncolada


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,The Whole Enchilada,"Moab, Utah",Black,34.1,1283,-7794,4.8,262,38.4855,-109.2320,...,Featured Ride,1,0,Moab,Utah,30+,0.905656,-7.604888,1.619591,3.515533
8313,Whole Uncolada,"Montrose, Colorado",Blue/Black,20.4,773,-3744,4.1,10,38.2338,-107.9942,...,Featured Ride,1,0,Montrose,Colorado,20-25,0.352891,-3.394507,0.834736,1.910235
16110,Thunder Monkey,"Boise, Idaho",Blue/Black,15.6,1170,-3864,5.0,11,43.7638,-116.1040,...,Featured Ride,1,0,Boise,Idaho,15-20,0.783180,-3.519259,0.834736,1.347795
4551,Cannell Trail - IMBA EPIC,"Kernville, California",Black,25.5,1718,-8030,4.6,43,35.9905,-118.3657,...,Featured Ride,1,0,Kernville,California,25-30,1.377131,-7.850233,1.619591,2.507828
3319,Nordhoff Ridge Road,"Ojai, California",Blue/Black,14.9,1183,-3523,5.0,1,34.5000,-119.1150,...,Trail,0,1,Ojai,California,10-15,0.797270,-3.164755,0.834736,1.265773
1332,CDO,"Catalina, Arizona",Black,21.1,1105,-6939,3.6,17,32.4407,-110.7861,...,Featured Ride,1,0,Catalina,Arizona,20-25,0.712730,-6.716029,1.619591,1.992258
10089,Big Boulder from Packer Saddle,"Downieville, California",Black,21.8,1644,-5813,4.3,3,39.6145,-120.6666,...,Featured Ride,1,0,Downieville,California,20-25,1.296926,-5.545440,1.619591,2.074280
8132,Never Eat Soggy Waffels,"Junction, Utah",Black,22.3,1778,-6709,4.0,1,38.3790,-112.4002,...,Featured Ride,1,0,Junction,Utah,20-25,1.442162,-6.476921,1.619591,2.132868
8676,Reddish Knob to Wild Oak: Chestnut & Grindston...,"Franklin, West Virginia",Blue/Black,13.3,1091,-3859,4.7,3,38.4625,-79.2418,...,Featured Ride,1,0,Franklin,West Virginia,10-15,0.697556,-3.514061,0.834736,1.078293
5660,Upper Alabama Hills Trail,"Lone Pine, California",Blue,5.8,435,-866,3.0,1,36.5948,-118.1526,...,Trail,0,1,Lone Pine,California,5-10,-0.013451,-0.402537,0.049881,0.199481


In [63]:
def cosim(v1, v2):
    dot_product = sum(n1 * n2 for n1, n2 in zip(v1, v2) )
    magnitude1 = math.sqrt(sum(n ** 2 for n in v1))
    magnitude2 = math.sqrt(sum(n ** 2 for n in v2))
    return dot_product / (magnitude1 * magnitude2)

In [64]:
v1 = [0.90565559, -7.60488757,  1.6195909,   3.51553271,  1.26812186,  2.6075075, -2.6075075]
v2 = [1.37713107, -7.85023322, 1.6195909, 2.50782783, 1.06278865, 2.6075075, -2.6075075]
cosim(v1, v2)

0.9924311777433537

In [65]:
euclidean_dist_recommendations("The Whole Enchilada", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.90565559 -7.60488757  1.6195909   3.51553271]]
The sorted euclidean distance measurements are:[  0.           1.13927728   1.40343582 ...,  36.9016584   37.29671653
  47.82939655]
The trail most similiar to the original trail has features: [ 1.37713107 -7.85023322  1.6195909   2.50782783]
The trail most similar to the original trail is: Cannell Trail - IMBA EPIC


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,The Whole Enchilada,"Moab, Utah",Black,34.1,1283,-7794,4.8,262,38.4855,-109.2320,...,Featured Ride,1,0,Moab,Utah,30+,0.905656,-7.604888,1.619591,3.515533
4551,Cannell Trail - IMBA EPIC,"Kernville, California",Black,25.5,1718,-8030,4.6,43,35.9905,-118.3657,...,Featured Ride,1,0,Kernville,California,25-30,1.377131,-7.850233,1.619591,2.507828
2933,Bike For Bender Hell Ride,"Big Bear City, California",Black,35.9,2349,-7055,4.8,4,34.1707,-116.8299,...,Featured Ride,1,0,Big Bear City,California,30+,2.061041,-6.836623,1.619591,3.726448
1332,CDO,"Catalina, Arizona",Black,21.1,1105,-6939,3.6,17,32.4407,-110.7861,...,Featured Ride,1,0,Catalina,Arizona,20-25,0.712730,-6.716029,1.619591,1.992258
8132,Never Eat Soggy Waffels,"Junction, Utah",Black,22.3,1778,-6709,4.0,1,38.3790,-112.4002,...,Featured Ride,1,0,Junction,Utah,20-25,1.442162,-6.476921,1.619591,2.132868
6811,CT - Segment 28: Champion Venture,"Rico, Colorado",Black,22.0,1924,-6550,4.5,13,37.4517,-108.0115,...,Trail,0,1,Rico,Colorado,20-25,1.600405,-6.311625,1.619591,2.097715
8361,Monarch Crest - IMBA EPIC,"Whitepine, Colorado",Blue/Black,36.2,2035,-5890,4.7,177,38.4965,-106.3254,...,Featured Ride,1,0,Whitepine,Colorado,30+,1.720712,-5.625489,0.834736,3.761600
15995,Paulina Peak to Horse Butte,"La Pine, Oregon",Blue/Black,38.7,1878,-5842,4.0,1,43.6895,-121.2545,...,Featured Ride,1,0,La Pine,Oregon,30+,1.550547,-5.575588,0.834736,4.054538
1088,Lemmon Drop,"Tanque Verde, Arizona",Black,18.8,1928,-6457,4.1,8,32.3980,-110.6893,...,Featured Ride,1,0,Tanque Verde,Arizona,15-20,1.604740,-6.214942,1.619591,1.722755
12,Haleakala Shuttle,"Kēōkea, Hawaii",Black,17.8,186,-7043,5.0,1,20.7086,-156.2540,...,Featured Ride,1,0,Kēōkea,Hawaii,15-20,-0.283330,-6.824148,1.619591,1.605580


In [66]:
def euc_dist(x,y):   
    dist = [(a - b)**2 for a, b in zip(x, y)]
    dist = math.sqrt(sum(dist))
    return dist

In [67]:
x = [0.90565559, -7.60488757,  1.6195909,   3.51553271,  1.26812186,  2.6075075, -2.6075075]
y = [1.37713107, -7.85023322,  1.6195909,   2.50782783,  1.06278865,  2.6075075, -2.6075075 ]
euc_dist(x,y)

1.1576331320931823

In [68]:
def list_to_npArray(vector1, vector2):
    '''convert the list to numpy array'''
    if type(vector1) == list:
        vector1 = np.array(vector1)
    if type(vector2) == list:
        vector2 = np.array(vector2)
    return vector1, vector2

def np_euc_dist(vector1,vector2): 
    vector1, vector2 = list_to_npArray(vector1, vector2)
    distance = np.linalg.norm(vector1-vector2, 2, 0)
    return distance
np_euc_dist(x,y)

1.1576331320931823

In [69]:
cos_sim_recommendations("Porcupine Rim", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.81027664 -2.58153942  1.6195909   1.242338  ]]
The sorted cosine similarity measurements are:[ 1.          0.99934477  0.99659743 ..., -0.93179636 -0.93182007
 -0.9319033 ]
The trail most similiar to the original trail has features: [ 0.94467425 -2.62728183  1.6195909   1.242338  ]
The trail most similar to the original trail is: Fore Mountain Trail


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Porcupine Rim,"Moab, Utah",Black,14.7,1195,-2962,4.7,240,38.5819,-109.4164,...,Featured Ride,1,0,Moab,Utah,10-15,0.810277,-2.581539,1.619591,1.242338
7977,Fore Mountain Trail,"Clifton Forge, Virginia",Black,14.7,1319,-3006,3.0,2,37.8986,-79.8364,...,Trail,0,1,Clifton Forge,Virginia,10-15,0.944674,-2.627282,1.619591,1.242338
7913,Whetstone Ridge Trail #523,"Nellysford, Virginia",Black,12.0,1091,-2740,5.0,1,37.8692,-79.1483,...,Trail,0,1,Nellysford,Virginia,10-15,0.697556,-2.350748,1.619591,0.925966
8895,Cache Creek Ridge Trail,"Clearlake, California",Blue/Black,9.9,979,-2060,2.5,2,39.0087,-122.4149,...,Trail,0,1,Clearlake,California,5-10,0.576165,-1.643820,0.834736,0.679898
6761,Thunder Mountain Trail #33098,"Panguitch, Utah",Blue/Black,8.1,689,-1681,4.6,59,37.7026,-112.2793,...,Trail,0,1,Panguitch,Utah,5-10,0.261848,-1.249812,0.834736,0.468983
8946,TRT-Flume-Chimney Beach Exit,"Incline Village, Nevada",Black,19.7,1485,-3735,5.0,3,39.3074,-119.9081,...,Featured Ride,1,0,Incline Village,Nevada,15-20,1.124594,-3.385150,1.619591,1.828213
17043,Warm Springs,"Cascade, Idaho",Double Black,14.5,1372,-3478,4.0,3,44.3057,-115.2588,...,Trail,0,1,Cascade,Idaho,10-15,1.002118,-3.117973,2.404446,1.218903
7703,Snow Mesa to Miner's Creek,"Lake City, Colorado",Black,16.7,1686,-3550,4.6,5,37.9397,-107.1587,...,Featured Ride,1,0,Lake City,Colorado,15-20,1.342448,-3.192824,1.619591,1.476688
1265,BLT to Noble Canyon,"Pine Valley, California",Black,20.2,1240,-3506,4.7,20,32.8570,-116.4241,...,Featured Ride,1,0,Pine Valley,California,20-25,0.859050,-3.147082,1.619591,1.886800
2686,Backbone Trail: Musch Camp to Will Rogers Trai...,"Topanga, California",Blue/Black,8.9,642,-1848,3.8,5,34.1040,-118.5725,...,Trail,0,1,Topanga,California,5-10,0.210907,-1.423425,0.834736,0.562723


In [70]:
euclidean_dist_recommendations("Porcupine Rim", 'Utah', Big_X)

The scaled features of the original trail are:[[ 0.81027664 -2.58153942  1.6195909   1.242338  ]]
The sorted euclidean distance measurements are:[  0.           0.14196861   0.40750713 ...,  40.86036643  41.41062747
  51.81477979]
The trail most similiar to the original trail has features: [ 0.94467425 -2.62728183  1.6195909   1.242338  ]
The trail most similar to the original trail is: Fore Mountain Trail


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Porcupine Rim,"Moab, Utah",Black,14.7,1195,-2962,4.7,240,38.5819,-109.4164,...,Featured Ride,1,0,Moab,Utah,10-15,0.810277,-2.581539,1.619591,1.242338
7977,Fore Mountain Trail,"Clifton Forge, Virginia",Black,14.7,1319,-3006,3.0,2,37.8986,-79.8364,...,Trail,0,1,Clifton Forge,Virginia,10-15,0.944674,-2.627282,1.619591,1.242338
7913,Whetstone Ridge Trail #523,"Nellysford, Virginia",Black,12.0,1091,-2740,5.0,1,37.8692,-79.1483,...,Trail,0,1,Nellysford,Virginia,10-15,0.697556,-2.350748,1.619591,0.925966
13769,Rock Creek Trail,"Centennial, Wyoming",Black,12.0,830,-3033,4.0,5,41.4562,-106.2727,...,Trail,0,1,Centennial,Wyoming,10-15,0.414671,-2.655351,1.619591,0.925966
8363,Monarch to Fooses Creek,"Salida, Colorado",Black,13.7,786,-3280,4.4,20,38.4960,-106.3247,...,Featured Ride,1,0,Salida,Colorado,10-15,0.366981,-2.912132,1.619591,1.125163
13752,White Pine to Bunchgrass,"Richmond, Utah",Black,10.7,886,-2663,3.6,5,41.8952,-111.6425,...,Featured Ride,1,0,Richmond,Utah,10-15,0.475366,-2.270699,1.619591,0.773638
18699,Idaho Trail Segment 17,"Grangeville, Idaho",Black,15.4,1795,-3128,3.0,1,45.8826,-115.2326,...,Featured Ride,1,0,Grangeville,Idaho,15-20,1.460588,-2.754113,1.619591,1.324360
3525,Ouachita National Recreation Trail - Segment 02,"Talihina, Oklahoma",Black,14.4,1614,-2455,4.0,2,34.7499,-94.7842,...,Trail,0,1,Talihina,Oklahoma,10-15,1.264410,-2.054462,1.619591,1.207186
16271,Grand Prize,"Sun Valley, Idaho",Black,13.0,1328,-2302,5.0,1,43.9346,-114.6912,...,Trail,0,1,Sun Valley,Idaho,10-15,0.954429,-1.895403,1.619591,1.043141
3478,West Spruce Trail #264,"Prescott, Arizona",Black,8.6,1180,-2790,3.0,1,34.5105,-112.5574,...,Trail,0,1,Prescott,Arizona,5-10,0.794019,-2.402728,1.619591,0.527571


In [71]:
cos_sim_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X)
#COMPARE Gunsight Ridge TO Boulder Lake Loop IN SAYING WHY COS SIM IS BETTER

The scaled features of the original trail are:[[ 2.39270009 -2.26758016  0.83473587  1.78134293]]
The sorted cosine similarity measurements are:[ 1.          0.99993482  0.99993339 ..., -0.95965306 -0.95973907
 -0.95984085]
The trail most similiar to the original trail has features: [ 2.27889567 -2.15114494  0.83473587  1.71103794]
The trail most similar to the original trail is: Gunsight Ridge


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Mount Falcon and Lair O' the Bear Loop,"Morrison, Colorado",Blue/Black,19.3,2655,-2660,3.6,43,39.6468,-105.1968,...,Featured Ride,1,0,Morrison,Colorado,15-20,2.392700,-2.267580,0.834736,1.781343
18165,Gunsight Ridge,"Government Camp, Oregon",Blue/Black,18.7,2550,-2548,4.8,8,45.3101,-121.6418,...,Featured Ride,1,0,Government Camp,Oregon,15-20,2.278896,-2.151145,0.834736,1.711038
18234,A Big Frenchy Pete,"McCall, Idaho",Double Black,47.3,6617,-6517,5.0,1,45.1847,-116.0040,...,Featured Ride,1,0,McCall,Idaho,30+,6.686920,-6.277318,2.404446,5.062243
9344,Doctor Park,"Gunnison, Colorado",Blue/Black,20.1,2734,-2719,4.7,127,38.7301,-106.7580,...,Featured Ride,1,0,Gunnison,Colorado,20-25,2.478324,-2.328917,0.834736,1.875083
19931,Coeur d'Alene River Loop,"Kellogg, Idaho",Black,34.7,4979,-4979,4.0,1,47.8832,-116.1312,...,Featured Ride,1,0,Kellogg,Idaho,30+,4.911571,-4.678413,1.619591,3.585838
17013,Peace Creek Loop,"Cascade, Idaho",Black,35.1,5065,-5052,5.0,1,44.3425,-115.7925,...,Featured Ride,1,0,Cascade,Idaho,30+,5.004783,-4.754304,1.619591,3.632708
7053,Potts Mountain Jeep Trail Loop,"New Castle, Virginia",Blue/Black,18.1,2475,-2472,3.0,1,37.6051,-80.0758,...,Featured Ride,1,0,New Castle,Virginia,15-20,2.197607,-2.072135,0.834736,1.640733
2756,Chino Hills Loop,"Yorba Linda, California",Blue/Black,20.0,2855,-2854,3.9,10,33.9090,-117.7801,...,Featured Ride,1,0,Yorba Linda,California,15-20,2.609470,-2.469263,0.834736,1.863365
8364,CDT Alpine Tunnel,"St. Elmo, Colorado",Blue/Black,18.0,2427,-2443,4.5,13,38.7059,-106.3406,...,Featured Ride,1,0,St. Elmo,Colorado,15-20,2.145582,-2.041987,0.834736,1.629015
7046,"Blue Ridge Parkway Epic: Glenwood, Mineshaft, ...","Blue Ridge, Virginia",Black,36.1,5234,-5234,5.0,1,37.4089,-79.7483,...,Featured Ride,1,0,Blue Ridge,Virginia,30+,5.187954,-4.943511,1.619591,3.749883


In [72]:
euclidean_dist_recommendations("Mount Falcon and Lair O' the Bear Loop", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 2.39270009 -2.26758016  0.83473587  1.78134293]]
The sorted euclidean distance measurements are:[  0.           0.1409995    0.16625476 ...,  39.79043036  40.37337069
  50.74949838]
The trail most similiar to the original trail has features: [ 2.47832438 -2.32891657  0.83473587  1.87508292]
The trail most similar to the original trail is: Doctor Park


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Mount Falcon and Lair O' the Bear Loop,"Morrison, Colorado",Blue/Black,19.3,2655,-2660,3.6,43,39.6468,-105.1968,...,Featured Ride,1,0,Morrison,Colorado,15-20,2.392700,-2.267580,0.834736,1.781343
9344,Doctor Park,"Gunnison, Colorado",Blue/Black,20.1,2734,-2719,4.7,127,38.7301,-106.7580,...,Featured Ride,1,0,Gunnison,Colorado,20-25,2.478324,-2.328917,0.834736,1.875083
18168,Boulder Lake Loop,"Government Camp, Oregon",Blue/Black,17.9,2676,-2674,3.7,7,45.3102,-121.6419,...,Featured Ride,1,0,Government Camp,Oregon,15-20,2.415461,-2.282135,0.834736,1.617298
18165,Gunsight Ridge,"Government Camp, Oregon",Blue/Black,18.7,2550,-2548,4.8,8,45.3101,-121.6418,...,Featured Ride,1,0,Government Camp,Oregon,15-20,2.278896,-2.151145,0.834736,1.711038
16120,Double D,"Boise, Idaho",Blue/Black,17.7,2596,-2594,4.0,6,43.6391,-116.1946,...,Featured Ride,1,0,Boise,Idaho,15-20,2.328753,-2.198967,0.834736,1.593863
8663,Braley's Pond & Confederate Breastworks Loop,"Weyers Cave, Virginia",Blue/Black,18.0,2753,-2758,4.0,16,38.2866,-79.3009,...,Featured Ride,1,0,Weyers Cave,Virginia,15-20,2.498918,-2.369461,0.834736,1.629015
19930,Bernard Peak - Scout Trail Option,"Spirit Lake, Idaho",Blue/Black,18.3,2782,-2786,4.3,3,47.9469,-116.6128,...,Featured Ride,1,0,Spirit Lake,Idaho,15-20,2.530349,-2.398570,0.834736,1.664168
18938,Buttercup Loop,"Hamilton, Montana",Blue/Black,21.1,2776,-2781,4.0,2,46.1538,-114.1289,...,Featured Ride,1,0,Hamilton,Montana,20-25,2.523846,-2.393372,0.834736,1.992258
2666,New Millennium Trail + Las Virgenes Downhill,"Agoura, California",Blue/Black,17.2,2558,-2568,4.1,22,34.1341,-118.7025,...,Featured Ride,1,0,Agoura,California,15-20,2.287566,-2.171937,0.834736,1.535275
9282,409s and Point Lookout Loop,"Crested Butte, Colorado",Blue/Black,16.9,2708,-2709,4.2,10,38.8644,-106.9130,...,Featured Ride,1,0,Crested Butte,Colorado,15-20,2.450144,-2.318521,0.834736,1.500123


In [73]:
cos_sim_recommendations("Enchanted Forest", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.43181738 -0.01684573  0.83473587 -0.30437181]]
The sorted cosine similarity measurements are:[ 1.          0.99983046  0.99979459 ..., -0.98803573 -0.9913864  -0.9915644 ]
The trail most similiar to the original trail has features: [-0.42423042 -0.00333093  0.83473587 -0.29265431]
The trail most similar to the original trail is: Deadline Trail


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Enchanted Forest,"Golden, Colorado",Blue/Black,1.5,49,-495,4.7,49,39.7173,-105.2483,...,Trail,0,1,Golden,Colorado,0-5,-0.431817,-0.016846,0.834736,-0.304372
10339,Deadline Trail,"Snowmass Village, Colorado",Blue/Black,1.6,56,-482,5.0,23,39.2199,-106.9101,...,Trail,0,1,Snowmass Village,Colorado,0-5,-0.424230,-0.003331,0.834736,-0.292654
11779,Rattlesnake Ridge Trail,"Shippensburg, Pennsylvania",Blue/Black,1.4,62,-503,3.0,2,40.0258,-77.3554,...,Trail,0,1,Shippensburg,Pennsylvania,0-5,-0.417727,-0.025163,0.834736,-0.316089
957,Vertigo Ridge,"El Paso, Texas",Blue/Black,1.3,36,-498,4.8,4,31.8035,-106.4925,...,Trail,0,1,El Paso,Texas,0-5,-0.445907,-0.019965,0.834736,-0.327807
19791,Summit Trail,"Mirrormont, Washington",Blue/Black,1.5,18,-510,4.3,6,47.4882,-121.9473,...,Trail,0,1,Mirrormont,Washington,0-5,-0.465417,-0.032440,0.834736,-0.304372
4352,Rocky Ridge Trail,"Etowah, North Carolina",Blue/Black,1.5,27,-464,4.5,13,35.2119,-82.5731,...,Trail,0,1,Etowah,North Carolina,0-5,-0.455662,0.015382,0.834736,-0.304372
17981,Douty Mill Trail,"Mifflinburg, Pennsylvania",Blue/Black,1.2,60,-507,3.0,1,40.9692,-77.2421,...,Trail,0,1,Mifflinburg,Pennsylvania,0-5,-0.419895,-0.029321,0.834736,-0.339524
4865,Ingles Field Gap,"Bent Creek, North Carolina",Blue/Black,1.7,39,-523,4.4,10,35.4976,-82.6481,...,Trail,0,1,Bent Creek,North Carolina,0-5,-0.442656,-0.045955,0.834736,-0.280937
8741,Middle Ridge Trail (Stinky Pete to Davis Creek),"Rand, West Virginia",Blue/Black,1.2,60,-477,4.0,3,38.2742,-81.6514,...,Trail,0,1,Rand,West Virginia,0-5,-0.419895,0.001867,0.834736,-0.339524
10784,Charit Creek Horse Trail,"Oneida, Tennessee",Blue/Black,1.1,34,-492,2.0,1,36.5287,-84.7291,...,Trail,0,1,Oneida,Tennessee,0-5,-0.448075,-0.013727,0.834736,-0.351242


In [74]:
euclidean_dist_recommendations("Enchanted Forest", 'Colorado', Big_X)
#it seems that euclidean is more likely to give back trails with shorter distances but greater absolute value in sum of ascent and descent

The scaled features of the original trail are:[[-0.43181738 -0.01684573  0.83473587 -0.30437181]]
The sorted euclidean distance measurements are:[  0.00000000e+00   1.94296590e-02   2.01245890e-02 ...,   4.38847610e+01
   4.44908958e+01   5.48732607e+01]
The trail most similiar to the original trail has features: [-0.42423042 -0.00333093  0.83473587 -0.29265431]
The trail most similar to the original trail is: Deadline Trail


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Enchanted Forest,"Golden, Colorado",Blue/Black,1.5,49,-495,4.7,49,39.7173,-105.2483,...,Trail,0,1,Golden,Colorado,0-5,-0.431817,-0.016846,0.834736,-0.304372
10339,Deadline Trail,"Snowmass Village, Colorado",Blue/Black,1.6,56,-482,5.0,23,39.2199,-106.9101,...,Trail,0,1,Snowmass Village,Colorado,0-5,-0.424230,-0.003331,0.834736,-0.292654
11779,Rattlesnake Ridge Trail,"Shippensburg, Pennsylvania",Blue/Black,1.4,62,-503,3.0,2,40.0258,-77.3554,...,Trail,0,1,Shippensburg,Pennsylvania,0-5,-0.417727,-0.025163,0.834736,-0.316089
957,Vertigo Ridge,"El Paso, Texas",Blue/Black,1.3,36,-498,4.8,4,31.8035,-106.4925,...,Trail,0,1,El Paso,Texas,0-5,-0.445907,-0.019965,0.834736,-0.327807
19791,Summit Trail,"Mirrormont, Washington",Blue/Black,1.5,18,-510,4.3,6,47.4882,-121.9473,...,Trail,0,1,Mirrormont,Washington,0-5,-0.465417,-0.032440,0.834736,-0.304372
4865,Ingles Field Gap,"Bent Creek, North Carolina",Blue/Black,1.7,39,-523,4.4,10,35.4976,-82.6481,...,Trail,0,1,Bent Creek,North Carolina,0-5,-0.442656,-0.045955,0.834736,-0.280937
17981,Douty Mill Trail,"Mifflinburg, Pennsylvania",Blue/Black,1.2,60,-507,3.0,1,40.9692,-77.2421,...,Trail,0,1,Mifflinburg,Pennsylvania,0-5,-0.419895,-0.029321,0.834736,-0.339524
4352,Rocky Ridge Trail,"Etowah, North Carolina",Blue/Black,1.5,27,-464,4.5,13,35.2119,-82.5731,...,Trail,0,1,Etowah,North Carolina,0-5,-0.455662,0.015382,0.834736,-0.304372
8741,Middle Ridge Trail (Stinky Pete to Davis Creek),"Rand, West Virginia",Blue/Black,1.2,60,-477,4.0,3,38.2742,-81.6514,...,Trail,0,1,Rand,West Virginia,0-5,-0.419895,0.001867,0.834736,-0.339524
10784,Charit Creek Horse Trail,"Oneida, Tennessee",Blue/Black,1.1,34,-492,2.0,1,36.5287,-84.7291,...,Trail,0,1,Oneida,Tennessee,0-5,-0.448075,-0.013727,0.834736,-0.351242


In [75]:
cos_sim_recommendations("Hermosa Creek Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.79618657 -1.9276309   0.83473587  1.69932044]]
The sorted cosine similarity measurements are:[ 1.          0.98776012  0.98549341 ..., -0.93076784 -0.93080369
 -0.93095376]
The trail most similiar to the original trail has features: [ 1.07473645 -2.27693656  0.83473587  1.46497047]
The trail most similar to the original trail is: Armstrong Pass Ride


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Hermosa Creek Trail,"Rico, Colorado",Blue/Black,18.6,1182,-2333,4.1,38,37.6299,-107.9158,...,Trail,0,1,Rico,Colorado,15-20,0.796187,-1.927631,0.834736,1.699320
8947,Armstrong Pass Ride,"South Lake Tahoe, California",Blue/Black,16.6,1439,-2669,4.7,7,38.7913,-119.9325,...,Featured Ride,1,0,South Lake Tahoe,California,15-20,1.074736,-2.276937,0.834736,1.464970
18170,Surveyor's Ridge Trail,"Odell, Oregon",Blue/Black,12.4,914,-1768,4.6,8,45.4088,-121.5496,...,Trail,0,1,Odell,Oregon,10-15,0.505714,-1.340257,0.834736,0.972836
8182,Mag 7,"Moab, Utah",Black,21.9,1815,-3849,4.6,119,38.5733,-109.7924,...,Featured Ride,1,0,Moab,Utah,20-25,1.482265,-3.503665,1.619591,2.085998
18598,Falls Creek Shuttle,"Stevenson, Washington",Blue/Black,15.4,1032,-2744,4.4,5,45.9900,-121.9131,...,Featured Ride,1,0,Stevenson,Washington,15-20,0.633609,-2.354907,0.834736,1.324360
1265,BLT to Noble Canyon,"Pine Valley, California",Black,20.2,1240,-3506,4.7,20,32.8570,-116.4241,...,Featured Ride,1,0,Pine Valley,California,20-25,0.859050,-3.147082,1.619591,1.886800
12134,The Divide Trail/ Mountain View,"Steamboat Springs, Colorado",Blue/Black,25.9,1824,-4476,4.5,36,40.3996,-106.6181,...,Featured Ride,1,0,Steamboat Springs,Colorado,25-30,1.492019,-4.155494,0.834736,2.554698
15995,Paulina Peak to Horse Butte,"La Pine, Oregon",Blue/Black,38.7,1878,-5842,4.0,1,43.6895,-121.2545,...,Featured Ride,1,0,La Pine,Oregon,30+,1.550547,-5.575588,0.834736,4.054538
8946,TRT-Flume-Chimney Beach Exit,"Incline Village, Nevada",Black,19.7,1485,-3735,5.0,3,39.3074,-119.9081,...,Featured Ride,1,0,Incline Village,Nevada,15-20,1.124594,-3.385150,1.619591,1.828213
8308,Tabeguache Trail - Roubideau Trail Section,"Montrose, Colorado",Blue/Black,22.1,2190,-3470,4.0,1,38.3625,-108.2505,...,Trail,0,1,Montrose,Colorado,20-25,1.888709,-3.109656,0.834736,2.109433


In [76]:
euclidean_dist_recommendations("Hermosa Creek Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.79618657 -1.9276309   0.83473587  1.69932044]]
The sorted euclidean distance measurements are:[  0.           0.50450411   0.59126221 ...,  40.90421871  41.51490245
  51.90748782]
The trail most similiar to the original trail has features: [ 1.07473645 -2.27693656  0.83473587  1.46497047]
The trail most similar to the original trail is: Armstrong Pass Ride


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Hermosa Creek Trail,"Rico, Colorado",Blue/Black,18.6,1182,-2333,4.1,38,37.6299,-107.9158,...,Trail,0,1,Rico,Colorado,15-20,0.796187,-1.927631,0.834736,1.699320
8947,Armstrong Pass Ride,"South Lake Tahoe, California",Blue/Black,16.6,1439,-2669,4.7,7,38.7913,-119.9325,...,Featured Ride,1,0,South Lake Tahoe,California,15-20,1.074736,-2.276937,0.834736,1.464970
18598,Falls Creek Shuttle,"Stevenson, Washington",Blue/Black,15.4,1032,-2744,4.4,5,45.9900,-121.9131,...,Featured Ride,1,0,Stevenson,Washington,15-20,0.633609,-2.354907,0.834736,1.324360
937,AZT - Passage 6 - Las Colinas,"Corona de Tucson, Arizona",Blue/Black,13.3,1295,-2429,2.7,3,31.8106,-110.7235,...,Trail,0,1,Corona de Tucson,Arizona,10-15,0.918662,-2.027433,0.834736,1.078293
12747,North Fork to Coulton Creek Shuttle,"Steamboat Springs, Colorado",Blue/Black,15.4,1682,-2072,4.6,5,40.7721,-106.7741,...,Featured Ride,1,0,Steamboat Springs,Colorado,15-20,1.338112,-1.656295,0.834736,1.324360
8376,CT - Segment 13,"Buena Vista, Colorado",Blue/Black,16.0,1706,-2672,4.0,2,38.8129,-106.2799,...,Trail,0,1,Buena Vista,Colorado,15-20,1.364125,-2.280055,0.834736,1.394665
12033,Coyote Trail,"Heber, Utah",Blue/Black,16.6,1837,-2159,3.4,5,40.5738,-111.4177,...,Trail,0,1,Heber,Utah,15-20,1.506109,-1.746740,0.834736,1.464970
11991,Wasatch Crest,"Alta, Utah",Blue/Black,12.5,1035,-2595,4.7,44,40.6138,-111.5710,...,Trail,0,1,Alta,Utah,10-15,0.636860,-2.200006,0.834736,0.984553
9197,Zion Curtain,"Moab, Utah",Blue/Black,17.9,1676,-1677,3.5,11,39.1956,-109.0513,...,Featured Ride,1,0,Moab,Utah,15-20,1.331609,-1.245653,0.834736,1.617298
20681,Wickersham Creek Trail,"Ester, Alaska",Blue,19.8,1097,-1977,3.0,1,65.1781,-148.0761,...,Trail,0,1,Ester,Alaska,15-20,0.704059,-1.557533,0.049881,1.839930


In [77]:
#SHOW HOW TO CALCULATE BOTH AND MENTION THAT IT MIGHT BE A DIFFERENCE OF OPINION BUT YOU THINK THAT GETTING A 
#SIMILAR DISTANCE AND PROPORTION OF CLIMBING TO DESCENDING IS BETTER THAN A SHORTER DISTANCE AND MORE CLIMBING/DESCENDING 
cos_sim_recommendations("Dakota Ridge Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.28658126  0.08399549  1.6195909  -0.29265431]]
The sorted cosine similarity measurements are:[ 1.          0.99983196  0.99972309 ..., -0.99803575 -0.99890167
 -0.99943111]
The trail most similiar to the original trail has features: [-0.46108138  0.1390943   2.40444594 -0.40982929]
The trail most similar to the original trail is: Spooky Pete's


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Dakota Ridge Trail,"Cañon City, Colorado",Black,1.6,183,-398,4.0,1,38.4621,-105.2516,...,Trail,0,1,Cañon City,Colorado,0-5,-0.286581,0.083995,1.619591,-0.292654
20237,Spooky Pete's,"Whitefish, Montana",Double Black,0.6,22,-345,5.0,1,48.3863,-114.4139,...,Trail,0,1,Whitefish,Montana,0-5,-0.461081,0.139094,2.404446,-0.409829
19897,Commitment Issues,"Millwood, Washington",Double Black,0.5,2,-352,4.0,1,47.6932,-117.3355,...,Trail,0,1,Millwood,Washington,0-5,-0.482758,0.131817,2.404446,-0.421547
7236,Alternate Flight Pattern,"West Lynchburg, Virginia",Black,1.4,200,-424,3.0,1,37.3582,-79.1614,...,Trail,0,1,West Lynchburg,Virginia,0-5,-0.268156,0.056966,1.619591,-0.316089
2012,Stair Steps,"Aliso Viejo, California",Double Black,0.4,0,-381,3.8,6,33.5800,-117.7549,...,Trail,0,1,Aliso Viejo,California,0-5,-0.484926,0.101669,2.404446,-0.433264
18173,Pro Line,"Government Camp, Oregon",Double Black,0.4,0,-382,5.0,2,45.2940,-121.7783,...,Trail,0,1,Government Camp,Oregon,0-5,-0.484926,0.100629,2.404446,-0.433264
5164,Zipline Trail,"Los Alamos, New Mexico",Double Black,0.5,0,-336,3.0,1,35.8805,-106.2680,...,Trail,0,1,Los Alamos,New Mexico,0-5,-0.484926,0.148451,2.404446,-0.421547
3473,Centennial Trail,"Prescott, Arizona",Black,1.8,200,-431,3.0,2,34.5581,-112.5045,...,Trail,0,1,Prescott,Arizona,0-5,-0.268156,0.049689,1.619591,-0.269219
10459,Wild Thing,"Keystone, Colorado",Double Black,0.4,2,-324,5.0,2,39.5988,-105.9446,...,Trail,0,1,Keystone,Colorado,0-5,-0.482758,0.160926,2.404446,-0.433264
15489,W250,"Marsing, Idaho",Double Black,0.7,28,-313,4.0,1,43.3590,-116.7386,...,Trail,0,1,Marsing,Idaho,0-5,-0.454578,0.172362,2.404446,-0.398112


In [78]:
euclidean_dist_recommendations("Dakota Ridge Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.28658126  0.08399549  1.6195909  -0.29265431]]
The sorted euclidean distance measurements are:[  0.00000000e+00   4.02404883e-02   4.54494699e-02 ...,   4.38738250e+01
   4.44839358e+01   5.48327632e+01]
The trail most similiar to the original trail has features: [-0.26815578  0.05696588  1.6195909  -0.31608931]
The trail most similar to the original trail is: Alternate Flight Pattern


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Dakota Ridge Trail,"Cañon City, Colorado",Black,1.6,183,-398,4.0,1,38.4621,-105.2516,...,Trail,0,1,Cañon City,Colorado,0-5,-0.286581,0.083995,1.619591,-0.292654
7236,Alternate Flight Pattern,"West Lynchburg, Virginia",Black,1.4,200,-424,3.0,1,37.3582,-79.1614,...,Trail,0,1,West Lynchburg,Virginia,0-5,-0.268156,0.056966,1.619591,-0.316089
3473,Centennial Trail,"Prescott, Arizona",Black,1.8,200,-431,3.0,2,34.5581,-112.5045,...,Trail,0,1,Prescott,Arizona,0-5,-0.268156,0.049689,1.619591,-0.269219
16597,Burro Trail,"Coal Creek, Colorado",Black,1.5,230,-394,2.0,3,39.8555,-105.3623,...,Trail,0,1,Coal Creek,Colorado,0-5,-0.235640,0.088154,1.619591,-0.304372
6499,South Leaf Trail,"Woodside, California",Black,1.6,178,-347,4.0,7,37.4018,-122.3334,...,Trail,0,1,Woodside,California,0-5,-0.292001,0.137015,1.619591,-0.292654
5264,North Bench Trail,"Jasper, Arkansas",Black,1.4,139,-403,4.0,1,35.8557,-93.5079,...,Trail,0,1,Jasper,Arkansas,0-5,-0.334271,0.078797,1.619591,-0.316089
9090,Pines Campground to Buck Meadows,"Tuolumne City, California",Black,1.7,147,-364,2.3,3,37.8190,-120.0951,...,Trail,0,1,Tuolumne City,California,0-5,-0.325600,0.119342,1.619591,-0.280937
7013,Hell Creek Road,"Beattyville, Kentucky",Black,1.7,208,-352,5.0,1,37.6373,-83.6572,...,Trail,0,1,Beattyville,Kentucky,0-5,-0.259485,0.131817,1.619591,-0.280937
4593,Central Spine,"Jemez Pueblo, New Mexico",Black,1.6,226,-350,3.4,5,35.5056,-106.8389,...,Trail,0,1,Jemez Pueblo,New Mexico,0-5,-0.239976,0.133896,1.619591,-0.292654
1160,Upper Sunset Trail,"Westway, Texas",Black,1.3,164,-463,2.0,4,31.9310,-106.5144,...,Trail,0,1,Westway,Texas,0-5,-0.307174,0.016421,1.619591,-0.327807


In [79]:
cos_sim_recommendations("Centennial Cone", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 2.12173717 -2.00664051  0.04988084  1.53527546]]
The sorted cosine similarity measurements are:[ 1.          0.99999158  0.99997993 ..., -0.99636578 -0.99639765
 -0.99644461]
The trail most similiar to the original trail has features: [ 2.77963514 -2.63351943  0.04988084  2.0039754 ]
The trail most similar to the original trail is: Loch Leven Loop


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Centennial Cone,"Idaho Springs, Colorado",Blue,17.2,2405,-2409,4.0,208,39.7372,-105.3710,...,Featured Ride,1,0,Idaho Springs,Colorado,15-20,2.121737,-2.006641,0.049881,1.535275
2941,Loch Leven Loop,"Yucaipa, California",Blue,21.2,3012,-3012,4.0,7,34.1043,-116.9962,...,Featured Ride,1,0,Yucaipa,California,20-25,2.779635,-2.633519,0.049881,2.003975
1243,Hollenbeck tour,"Jamul, California",Blue,14.2,1975,-1976,4.0,1,32.6706,-116.8230,...,Trail,0,1,Jamul,California,10-15,1.655681,-1.556494,0.049881,1.183751
10397,Village to Village Tour,"Avon, Colorado",Blue,14.6,2046,-2048,2.7,3,39.6346,-106.5383,...,Featured Ride,1,0,Avon,Colorado,10-15,1.732634,-1.631345,0.049881,1.230620
1991,Ridge Rollercoaster Romance,"Laguna Beach, California",Blue,15.9,2177,-2179,3.1,9,33.5507,-117.7967,...,Featured Ride,1,0,Laguna Beach,California,15-20,1.874619,-1.767532,0.049881,1.382948
11944,Corner Canyon Front and Back,"Draper, Utah",Blue,16.4,2333,-2333,5.0,1,40.5060,-111.8455,...,Featured Ride,1,0,Draper,Utah,15-20,2.043700,-1.927631,0.049881,1.441535
10434,Rock Island to Galena Ditch,"Breckenridge, Colorado",Blue,18.5,2545,-2552,4.0,8,39.4870,-106.0452,...,Featured Ride,1,0,Breckenridge,Colorado,15-20,2.273476,-2.155303,0.049881,1.687603
10472,Lair O' the Bear - Bear Creek Trail,"Genesee, Colorado",Blue,12.2,1673,-1673,4.0,190,39.6676,-105.2570,...,Featured Ride,1,0,Genesee,Colorado,10-15,1.328358,-1.241495,0.049881,0.949401
16262,Curly's Loop,"Sun Valley, Idaho",Blue,12.1,1657,-1658,4.0,6,43.7828,-114.5565,...,Featured Ride,1,0,Sun Valley,Idaho,10-15,1.311016,-1.225901,0.049881,0.937683
12135,Beall - Ridge - Rotary Loop,"Steamboat Springs, Colorado",Blue,15.8,2169,-2152,4.5,28,40.4539,-106.9166,...,Featured Ride,1,0,Steamboat Springs,Colorado,15-20,1.865948,-1.739463,0.049881,1.371230


In [80]:
euclidean_dist_recommendations("Centennial Cone", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 2.12173717 -2.00664051  0.04988084  1.53527546]]
The sorted euclidean distance measurements are:[  0.           0.11134936   0.11585111 ...,  40.22896479  40.81417156
  51.21569953]
The trail most similiar to the original trail has features: [ 2.19110368 -2.07109572  0.04988084  1.47668797]
The trail most similar to the original trail is: Jackson Demo Forest


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Centennial Cone,"Idaho Springs, Colorado",Blue,17.2,2405,-2409,4.0,208,39.7372,-105.3710,...,Featured Ride,1,0,Idaho Springs,Colorado,15-20,2.121737,-2.006641,0.049881,1.535275
10069,Jackson Demo Forest,"Fort Bragg, California",Blue,16.7,2469,-2471,4.3,4,39.3355,-123.7566,...,Featured Ride,1,0,Fort Bragg,California,15-20,2.191104,-2.071096,0.049881,1.476688
7331,Golden Gate to Tennessee Beach Loop,"Sausalito, California",Blue,17.9,2443,-2477,4.2,9,37.8330,-122.4829,...,Featured Ride,1,0,Sausalito,California,15-20,2.162924,-2.077333,0.049881,1.617298
11944,Corner Canyon Front and Back,"Draper, Utah",Blue,16.4,2333,-2333,5.0,1,40.5060,-111.8455,...,Featured Ride,1,0,Draper,Utah,15-20,2.043700,-1.927631,0.049881,1.441535
7330,Mount Tamalpais Summit Loop,"Mill Valley, California",Blue,16.2,2469,-2468,4.0,16,37.9186,-122.5538,...,Featured Ride,1,0,Mill Valley,California,15-20,2.191104,-2.067977,0.049881,1.418100
15575,Game Creek Loop,"Jackson, Wyoming",Blue,18.5,2435,-2435,4.2,18,43.4695,-110.7722,...,Featured Ride,1,0,Jackson,Wyoming,15-20,2.154253,-2.033670,0.049881,1.687603
6104,Shaver Lake Loop,"Auberry, California",Blue,18.4,2355,-2355,3.3,7,37.1074,-119.3179,...,Featured Ride,1,0,Auberry,California,15-20,2.067545,-1.950502,0.049881,1.675885
10335,Hay Park,"El Jebel, Colorado",Blue,18.6,2365,-2364,3.8,6,39.3036,-107.1246,...,Featured Ride,1,0,El Jebel,Colorado,15-20,2.078383,-1.959859,0.049881,1.699320
2743,Lower Monroe Truck Trail,"Glendora, California",Blue,16.1,2319,-2319,4.1,45,34.1562,-117.8367,...,Featured Ride,1,0,Glendora,California,15-20,2.028526,-1.913076,0.049881,1.406383
6531,Monte Bello-Stevens Creek Loop,"Cupertino, California",Blue,17.3,2536,-2538,3.7,18,37.2945,-122.0819,...,Featured Ride,1,0,Cupertino,California,15-20,2.263722,-2.140749,0.049881,1.546993


In [81]:
cos_sim_recommendations("Mustang Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.30609059 -0.33392379  0.83473587 -0.22234932]]
The sorted cosine similarity measurements are:[ 1.          0.99928511  0.99746807 ..., -0.99076708 -0.99166373
 -0.99374527]
The trail most similiar to the original trail has features: [-0.32126451 -0.32768619  0.83473587 -0.25750181]
The trail most similar to the original trail is: Barlow Road


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Mustang Trail,"Golden, Colorado",Blue/Black,2.2,165,-800,3.9,35,39.8159,-105.2926,...,Trail,0,1,Golden,Colorado,0-5,-0.306091,-0.333924,0.834736,-0.222349
6011,Barlow Road,"Almaden Valley, California",Blue/Black,1.9,151,-794,4.0,2,37.1572,-121.8811,...,Trail,0,1,Almaden Valley,California,0-5,-0.321265,-0.327686,0.834736,-0.257502
7668,East Fork Bear Creek Trail #513,"Telluride, Colorado",Black,1.2,0,-1098,5.0,1,37.8885,-107.8071,...,Trail,0,1,Telluride,Colorado,0-5,-0.484926,-0.643725,1.619591,-0.339524
3302,Lower Romero,"Summerland, California",Black,1.3,0,-1055,4.3,9,34.4720,-119.5934,...,Trail,0,1,Summerland,California,0-5,-0.484926,-0.599022,1.619591,-0.327807
12054,Holman Ridge - Cabin Flat #172,"Alta, Utah",Black,1.3,0,-1049,3.0,2,40.4859,-111.6123,...,Trail,0,1,Alta,Utah,0-5,-0.484926,-0.592784,1.619591,-0.327807
15548,Trail #330,"Mountain Home, Idaho",Black,1.1,0,-1162,3.0,1,43.4793,-115.7146,...,Trail,0,1,Mountain Home,Idaho,0-5,-0.484926,-0.710259,1.619591,-0.351242
4619,Saddleback Trail #232,"Santa Fe, New Mexico",Blue/Black,2.5,232,-804,4.3,7,35.7427,-105.8581,...,Trail,0,1,Santa Fe,New Mexico,0-5,-0.233473,-0.338082,0.834736,-0.187197
5732,World Cup,"Angel Fire, New Mexico",Black,1.4,0,-1076,5.0,2,36.3720,-105.2460,...,Trail,0,1,Angel Fire,New Mexico,0-5,-0.484926,-0.620853,1.619591,-0.316089
4080,Capulin Snow Play Downhill Trail,"Sandia Heights, New Mexico",Black,1.5,0,-1080,3.5,2,35.2097,-106.4354,...,Trail,0,1,Sandia Heights,New Mexico,0-5,-0.484926,-0.625012,1.619591,-0.304372
18977,White Stallion Camp Trail #288,"Hamilton, Montana",Black,1.4,0,-1150,3.0,1,46.0616,-113.9920,...,Trail,0,1,Hamilton,Montana,0-5,-0.484926,-0.697784,1.619591,-0.316089


In [82]:
euclidean_dist_recommendations("Mustang Trail", 'Colorado', Big_X)

The scaled features of the original trail are:[[-0.30609059 -0.33392379  0.83473587 -0.22234932]]
The sorted euclidean distance measurements are:[  0.00000000e+00   3.87924421e-02   8.07859718e-02 ...,   4.35908980e+01
   4.41850070e+01   5.45766126e+01]
The trail most similiar to the original trail has features: [-0.32126451 -0.32768619  0.83473587 -0.25750181]
The trail most similar to the original trail is: Barlow Road


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,Mustang Trail,"Golden, Colorado",Blue/Black,2.2,165,-800,3.9,35,39.8159,-105.2926,...,Trail,0,1,Golden,Colorado,0-5,-0.306091,-0.333924,0.834736,-0.222349
6011,Barlow Road,"Almaden Valley, California",Blue/Black,1.9,151,-794,4.0,2,37.1572,-121.8811,...,Trail,0,1,Almaden Valley,California,0-5,-0.321265,-0.327686,0.834736,-0.257502
4619,Saddleback Trail #232,"Santa Fe, New Mexico",Blue/Black,2.5,232,-804,4.3,7,35.7427,-105.8581,...,Trail,0,1,Santa Fe,New Mexico,0-5,-0.233473,-0.338082,0.834736,-0.187197
7074,Glenwood Horse Trail (3004E) - Onion Mountain,"Buchanan, Virginia",Blue/Black,2.1,244,-839,5.0,2,37.5073,-79.5132,...,Trail,0,1,Buchanan,Virginia,0-5,-0.220466,-0.374468,0.834736,-0.234067
6207,Little Purgatory Trail,"Leeds, Utah",Blue/Black,2.6,103,-849,4.0,1,37.2273,-113.3581,...,Trail,0,1,Leeds,Utah,0-5,-0.373289,-0.384864,0.834736,-0.175479
8986,Tahoe Mountain Trail,"South Lake Tahoe, California",Blue/Black,2.6,89,-773,4.3,6,38.9053,-120.0270,...,Trail,0,1,South Lake Tahoe,California,0-5,-0.388463,-0.305855,0.834736,-0.175479
5110,Water Canyon Trail,"Los Alamos, New Mexico",Blue/Black,2.5,167,-710,4.5,2,35.8447,-106.3869,...,Trail,0,1,Los Alamos,New Mexico,0-5,-0.303923,-0.240360,0.834736,-0.187197
13762,Bear Hollow Trail,"Richmond, Utah",Blue/Black,2.6,187,-887,3.5,2,41.8835,-111.6364,...,Trail,0,1,Richmond,Utah,0-5,-0.282246,-0.424369,0.834736,-0.175479
4252,Live Wire 2,"Chattanooga, Tennessee",Blue/Black,2.9,192,-741,4.3,6,35.0624,-85.3853,...,Trail,0,1,Chattanooga,Tennessee,0-5,-0.276827,-0.272587,0.834736,-0.140327
16368,Elk Flat Relay,"Driggs, Idaho",Blue/Black,3.0,172,-738,4.0,1,43.7031,-111.3411,...,Trail,0,1,Driggs,Idaho,0-5,-0.298504,-0.269469,0.834736,-0.128609


In [83]:
cos_sim_recommendations("North Table Loop", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.47211492 -0.47323022  0.04988084  0.3986781 ]]
The sorted cosine similarity measurements are:[ 1.          0.99975227  0.99966609 ..., -0.9919446  -0.99195808
 -0.99198416]
The trail most similiar to the original trail has features: [ 8.68771065 -8.2972613   0.83473587  7.27684973]
The trail most similar to the original trail is: Eagle Backcountry Tour


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,North Table Loop,"Golden, Colorado",Blue,7.5,883,-934,3.7,119,39.7812,-105.2301,...,Featured Ride,1,0,Golden,Colorado,5-10,0.472115,-0.473230,0.049881,0.398678
10338,Eagle Backcountry Tour,"Eagle, Colorado",Blue/Black,66.2,8463,-8460,3.0,3,39.6400,-106.8310,...,Featured Ride,1,0,Eagle,Colorado,30+,8.687711,-8.297261,0.834736,7.276850
3536,Ouachita National Recreation Trail - IMBA Epic,"Mena, Arkansas",Black,108.4,13381,-14826,4.0,3,34.6723,-94.3197,...,Trail,0,1,Mena,Arkansas,30+,14.018093,-14.915356,1.619591,12.221634
13991,Main White Trail,"Collinsville, Connecticut",Blue,9.6,1120,-1187,4.0,1,41.7605,-72.9509,...,Trail,0,1,Collinsville,Connecticut,5-10,0.728988,-0.736249,0.049881,0.644746
3701,Pinhoti - Mack White Gap to Narrows,"Summerville, Georgia",Blue,9.4,1091,-1136,3.7,3,34.4759,-85.2928,...,Trail,0,1,Summerville,Georgia,5-10,0.697556,-0.683229,0.049881,0.621311
6142,Precipice/Sidewinder/Barrel Roll loop,"Santa Clara, Utah",Blue,7.9,946,-952,4.2,20,37.1291,-113.6632,...,Featured Ride,1,0,Santa Clara,Utah,5-10,0.540398,-0.491943,0.049881,0.445548
12358,Frick Park Singletrack,"Edgewood, Pennsylvania",Blue,8.8,1081,-1083,3.7,9,40.4305,-79.9005,...,Featured Ride,1,0,Edgewood,Pennsylvania,5-10,0.686718,-0.628131,0.049881,0.551006
8320,Clear Creek Loop,"Montrose, Colorado",Blue,11.3,1403,-1416,3.0,2,38.2596,-108.0384,...,Featured Ride,1,0,Montrose,Colorado,10-15,1.035718,-0.974318,0.049881,0.843943
9196,Turkey Flats Loop,"Redlands, Colorado",Blue,9.5,1161,-1163,4.2,6,38.8620,-108.7729,...,Featured Ride,1,0,Redlands,Colorado,5-10,0.773426,-0.711299,0.049881,0.633028
10091,Old Miners' Loop from Town,"Nevada City, California",Blue,10.4,1266,-1272,3.8,6,39.2676,-121.0141,...,Featured Ride,1,0,Nevada City,California,10-15,0.887230,-0.824615,0.049881,0.738486


In [84]:
euclidean_dist_recommendations("North Table Loop", 'Colorado', Big_X)

The scaled features of the original trail are:[[ 0.47211492 -0.47323022  0.04988084  0.3986781 ]]
The sorted euclidean distance measurements are:[  0.00000000e+00   4.46162978e-02   6.31341456e-02 ...,   4.26879507e+01
   4.33022619e+01   5.36947588e+01]
The trail most similiar to the original trail has features: [ 0.48295343 -0.43684421  0.04988084  0.4221131 ]
The trail most similar to the original trail is: Triple C to Step Creek Double Loop


,name,location,difficulty,length,ascent,descent,stars,starVotes,latitude,longitude,...,category,type_Featured Ride,type_Trail,city/town,state,length_range,ascent_scaled,descent_scaled,difficulty_encoded_scaled,length_scaled
original,North Table Loop,"Golden, Colorado",Blue,7.5,883,-934,3.7,119,39.7812,-105.2301,...,Featured Ride,1,0,Golden,Colorado,5-10,0.472115,-0.473230,0.049881,0.398678
18507,Triple C to Step Creek Double Loop,"Banks, Oregon",Blue,7.7,893,-899,4.0,3,45.7070,-123.3375,...,Featured Ride,1,0,Banks,Oregon,5-10,0.482953,-0.436844,0.049881,0.422113
9272,Strand Hill,"Crested Butte, Colorado",Blue,7.5,874,-874,4.5,55,38.8644,-106.9130,...,Featured Ride,1,0,Crested Butte,Colorado,5-10,0.462360,-0.410854,0.049881,0.398678
9829,Watters Smith State Park,"Stonewood, West Virginia",Blue,7.6,876,-873,4.0,2,39.1719,-80.4074,...,Featured Ride,1,0,Stonewood,West Virginia,5-10,0.464528,-0.409815,0.049881,0.410396
20566,Hillside Park,"Anchorage, Alaska",Blue,7.1,887,-887,4.3,6,61.1386,-149.7467,...,Featured Ride,1,0,Anchorage,Alaska,5-10,0.476450,-0.424369,0.049881,0.351808
5275,Syllamo Trails - Bald Scrappy Loop,"Mountain View, Arkansas",Blue,7.1,927,-916,3.7,10,35.9627,-92.1189,...,Trail,0,1,Mountain View,Arkansas,5-10,0.519804,-0.454517,0.049881,0.351808
3764,Mountain Bike Trail,"Cleveland, Georgia",Blue,7.6,955,-914,3.6,7,34.7207,-83.7249,...,Trail,0,1,Cleveland,Georgia,5-10,0.550152,-0.452438,0.049881,0.410396
6142,Precipice/Sidewinder/Barrel Roll loop,"Santa Clara, Utah",Blue,7.9,946,-952,4.2,20,37.1291,-113.6632,...,Featured Ride,1,0,Santa Clara,Utah,5-10,0.540398,-0.491943,0.049881,0.445548
6154,City Creek to Rusty Cliffs Trail,"Saint George, Utah",Blue,8.2,895,-900,2.4,5,37.1259,-113.5827,...,Featured Ride,1,0,Saint George,Utah,5-10,0.485121,-0.437884,0.049881,0.480701
9273,Snodgrass,"Crested Butte, Colorado",Blue,7.3,855,-853,4.2,43,38.9228,-107.0046,...,Featured Ride,1,0,Crested Butte,Colorado,5-10,0.441767,-0.389023,0.049881,0.375243


In [85]:
US_trails.columns

Index(['name', 'location', 'difficulty', 'length', 'ascent', 'descent',
       'stars', 'starVotes', 'latitude', 'longitude', 'summary', 'url',
       'difficulty_encoded', 'category', 'type_Featured Ride', 'type_Trail',
       'city/town', 'state', 'length_range', 'ascent_scaled', 'descent_scaled',
       'difficulty_encoded_scaled', 'length_scaled'],
      dtype='object')

In [86]:
US_trails['length_range'].value_counts()

0-5      16279
5-10      2381
10-15      995
15-20      452
20-25      251
30+        221
25-30      112
Name: length_range, dtype: int64